# Model Training

In [1]:
import pandas as pd
import sys
sys.path.append('../')
from utils.machine_learning import preprocess_pipeline, evaluate_models, visualize_roc_curves, visualize_confusion_matrix, initiate_model_trainer

# Data

In [3]:
df_original = pd.read_csv("./Ml_Datasets/original.csv")
df_all = pd.read_csv("./Ml_Datasets/all_features.csv")
df_new = pd.read_csv("./Ml_Datasets/new_features.csv")

# Preprocessing the Data

In [4]:
X_train_original, X_test_original, y_train_original, y_test_original = preprocess_pipeline(df_original, test_size=0.3, target_name="target")
X_train_all, X_test_all, y_train_all, y_test_all = preprocess_pipeline(df_all, test_size=0.3, target_name="target")
X_train_new, X_test_new, y_train_new, y_test_new = preprocess_pipeline(df_new, test_size=0.3, target_name="target")

# Models

In [5]:
model_report = initiate_model_trainer((X_train_original, y_train_original, X_test_original, y_test_original), "test_experiment")

c:\Users\Vitor\anaconda3\envs\pred-env\Lib\site-packages\sklearn\linear_model\_logistic.py:1186: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
c:\Users\Vitor\anaconda3\envs\pred-env\Lib\site-packages\sklearn\linear_model\_logistic.py:1186: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
c:\Users\Vitor\anaconda3\envs\pred-env\Lib\site-packages\sklearn\linear_model\_logistic.py:1186: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
c:\Users\Vitor\anaconda3\envs\pred-env\Lib\site-packages\sklearn\linear_model\_logistic.py:1186: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
c:\Users\Vitor\anaconda3\envs\pred-env\Lib\site-packages\sklearn\linear_model\_logistic.py:1186: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
c:\Users\Vitor\anaconda3\envs\pred-env\Lib\site-pa

AttributeError: 'DAGsHubLogger' object has no attribute 'log_model'